In [1]:
import pandas as pd
import tensorflow as tf
tf.config.set_visible_devices([], 'GPU')

In [2]:

import NeuralNetwork.pretrained_models as pre_models
from NeuralNetwork.tools import predict, predict_mul
from NeuralNetwork.preprocessing import skewness_pure, min_max_normalization_pure


def predict_df(X_df):

    model_C_046, input_features_C_046 = pre_models.Input_C_046('weights/')
    
    outputs = ['Input_C_046']

    preprocessings = ['min_max_normalization']

    models = [model_C_046]

    input_features = [input_features_C_046]

    result = {}

    for i in range(len(outputs)):

            if preprocessings[i] == 'skewness':
                X = skewness_pure(X_df, input_features[i])
            elif preprocessings[i] == 'min_max_normalization':
                X = min_max_normalization_pure(X_df, input_features[i])

            result[outputs[i]] = predict_mul(models[i], X)
    
    result_df = pd.DataFrame(result, columns = outputs)
    return result_df


In [3]:
def nn_ave(x, y):

    rmse = []
    r2 = []
    for i in np.arange(10):
        X_train, X_test, y_train, y_test = train_test_split(
            x, y, test_size=0.2, random_state=i)

        nn_pred = predict_df(X_test)

        rmse.append(math.sqrt(mean_squared_error(y_test, nn_pred)))
        r2.append(r2_score(y_test, nn_pred))

    test_rmse = np.array(rmse).mean()
    print('test_rmse_ave:', test_rmse)
    print(rmse)
    print('\n')
    test_r2 = np.array(r2).mean()
    print('test_r2_ave:', test_r2)
    print(r2)

In [4]:
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
from sklearn.linear_model import LinearRegression, LassoCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import PowerTransformer
import math
import warnings
warnings.filterwarnings('ignore')

# 處理 data 套件
import numpy as np
import pandas as pd
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score,KFold
from scipy.stats import skew
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Lasso
from mlxtend.regressor import StackingRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.svm import SVR

In [5]:
from imbdpack import skew_pro,lr_rmse_ave,lr_rmse_ave_fea,laso_rmse_ave,ElasticNet_rmse_ave,xgb_ave,svr_rmse_ave,rand_ave
from imbdpack import voting_ave,stack_ave,ave

In [6]:
fea_number=16
data=pd.read_csv('./BayesianRidge_Pre_0/Input_C_046.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑
y=data.Predict
nn=data.drop(['Predict'], axis=1)
skew_data=skew_pro(data)
num=skew_data.select_dtypes(exclude='object')
numcorr=num.corr()
title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea_number+1).to_frame().index.to_numpy()[1:]
skew_x=skew_data.drop(['Predict'], axis=1)
title

array(['Input_C_043', 'Input_C_044', 'Input_C_041', 'Input_C_045',
       'Input_C_047', 'Input_C_042', 'Input_A4_005', 'Input_C_098',
       'Input_A4_004', 'Input_A6_009', 'Output_A5', 'Input_C_109',
       'Input_C_061', 'Output_A4', 'Input_C_108', 'Input_C_134'],
      dtype=object)

In [7]:
test=pd.read_csv('./final_testX.csv',index_col=0).drop(['Number'], axis=1)[title] ## 改路徑

In [8]:
nn_ave(nn, y)

test_rmse_ave: 0.001523958264884013
[0.0014125852948577398, 0.0023671363058867466, 0.0021276334129587803, 0.0012851579605113054, 0.001296815050648222, 0.0006880579686143848, 0.0014849525480496281, 0.0019389859527951967, 0.0022901706430295764, 0.00034808751148854777]


test_r2_ave: -21.918909581509187
[-15.332713512700355, -44.66809531253611, -36.60505440055801, -12.506381161593293, -15.098425850637927, -3.8905801082174127, -17.894937649967957, -26.720641030443094, -46.15401687200702, -0.31824991643069955]


## Linear Regression Baseline

In [4]:
lr_rmse_ave_fea(skew_data,16)

train_rmse: 0.00024638849647603324
test_rmse: 0.0002672612705032694
test_r2: 0.35916233971462896


## Lasso

In [5]:
laso_rmse_ave(skew_data[title],y,9e-06)

test_rmse_ave: 0.0002642850509281665
[0.00027097383936093485, 0.00031886849895088927, 0.0002550965733229793, 0.00024486634466860854, 0.00024093355243655855, 0.0002555353974679931, 0.00029177361706486855, 0.0003048328870649025, 0.00023510298004375183, 0.0002248668189001782]


test_r2_ave: 0.38474826414165114
[0.3989872129365518, 0.17131476560185022, 0.45941764367971494, 0.509674842213582, 0.444323672953874, 0.3254526260874956, 0.27052174606463075, 0.31486274716295537, 0.5030653545502292, 0.4498620301656273]


## ElasticNet

In [6]:
ElasticNet_rmse_ave(skew_data[title],y,[0.005],0.01)

test_rmse_ave: 0.0002814134872676714
[0.00029075191314107145, 0.00031907460094731706, 0.00028597286228006723, 0.00028000854591054734, 0.0002656054607267945, 0.00026008158501571773, 0.00029488495348064186, 0.00031821139991168416, 0.0002682924071140668, 0.00023125114414880636]


test_r2_ave: 0.3063021151850678
[0.3080509066256667, 0.17024317102045283, 0.32063641895122263, 0.3588368514040312, 0.3246928833924372, 0.30123760340028827, 0.25488117149713174, 0.253404424741836, 0.3528577241117127, 0.4181799967058991]


## SVR

In [7]:
model=SVR(kernel='rbf', C=1.25, gamma= 5e-06, epsilon= 0)
svr_rmse_ave(skew_data,16,model)

train_rmse: 0.00026912186154754635
test_rmse: 0.00028086127254813096
test_r2: 0.29523303291210157


## XGB

In [8]:
best_xgb_model =  XGBRegressor(
                objective ='reg:squarederror',
              learning_rate = 0.15,
              booster = 'gbtree', 
              n_estimators = 190, 
              max_depth = 1, 
              min_child_weight = 2,
              seed = 42,
              gamma = 0,
              subsample = 0.6,
              colsample_bytree = 0.6,
              reg_alpha =  0,
              reg_lambda = 1.5)
xgb_ave(skew_data[title], y,best_xgb_model)

test_rmse_ave: 0.00027155172419481076
[0.0002751923002569586, 0.00032564713585421805, 0.00027400394096301994, 0.00027277623111693583, 0.00025345674006030735, 0.0002483625613728319, 0.00028341774067188846, 0.00030909210711893516, 0.00025226024782844134, 0.00022130823670457142]


test_r2_ave: 0.3533839982354746
[0.38012868532006794, 0.13570720726212948, 0.3763136548073528, 0.39153019779956044, 0.38505678150757316, 0.362790004846688, 0.31170538051383545, 0.29558308867183014, 0.42788854412588606, 0.4671364374998227]


## RandomForest

In [9]:
rand_model = RandomForestRegressor( 
            criterion ='mse', 
            n_estimators = 40,
            max_depth = 9,
            min_samples_split = 6,
            max_leaf_nodes = 12,
            min_samples_leaf =1,
            random_state = 42)  
rand_ave(skew_x[title], y,rand_model)

test_rmse_ave: 0.00027348901382709347
[0.00026959501974372854, 0.0003497377986352029, 0.00025531020122186525, 0.00025706192554026227, 0.00025761516679060953, 0.0002561238011519452, 0.00030277165255769144, 0.0003119771575063276, 0.00025589713890538475, 0.00021880027621791703]


test_r2_ave: 0.34006544866855765
[0.4050880223724711, 0.003100197158012241, 0.4585118547335867, 0.4596172301770748, 0.36471268582436167, 0.3223425824072269, 0.2144917804003349, 0.2823717330389788, 0.4112731156478622, 0.4791452849256669]


In [9]:
lr = LinearRegression()
lasso_mod=Lasso(alpha=9e-06)
elastic_mod=ElasticNet(alpha=[0.005], l1_ratio=0.01)
svr_mod=SVR(kernel='rbf', C=1.25, gamma= 5e-06, epsilon= 0)

xgb_mod =  XGBRegressor(
                objective ='reg:squarederror',
              learning_rate = 0.15,
              booster = 'gbtree', 
              n_estimators = 190, 
              max_depth = 1, 
              min_child_weight = 2,
              seed = 42,
              gamma = 0,
              subsample = 0.6,
              colsample_bytree = 0.6,
              reg_alpha =  0,
              reg_lambda = 1.5)

random_mod = RandomForestRegressor( 
            criterion ='mse', 
            n_estimators = 40,
            max_depth = 9,
            min_samples_split = 6,
            max_leaf_nodes = 12,
            min_samples_leaf =1,
            random_state = 42) 
vote_mod = VotingRegressor([ ('Lasso', lasso_mod), ('SVR', svr_mod),('Elastic', elastic_mod), ('Linear', lr),
                            ('XGBRegressor', xgb_mod),('RandomForest', random_mod)])
# 
# vote_mod = VotingRegressor([ ('SVR', svr_mod), ('Linear', lr)
#                             ,('RandomForest', random_mod)])
voting_ave(skew_data[title], y,vote_mod)


test_rmse_ave: 0.0002656770609884469
[0.00027462133967079617, 0.00032380954549673615, 0.0002560408054777814, 0.00025554678343050287, 0.00024485685629289624, 0.0002471299264829049, 0.00028361046148841747, 0.0003033337005568665, 0.00024503025354990994, 0.0002227909374376569]


test_r2_ave: 0.37972273689781816
[0.38269819715212183, 0.14543390137185652, 0.45540833915079815, 0.46596856969821177, 0.42607933080545235, 0.3690993146099498, 0.31076899734626207, 0.3215852681238591, 0.46021299274551597, 0.4599724579741542]


In [10]:

stack_mod = StackingRegressor(regressors=[vote_mod, lr], 
                           meta_regressor=lasso_mod, use_features_in_secondary=True
                          )

stack_ave(skew_data[title], y,stack_mod)

test_rmse_ave: 0.0002642850509281665
[0.0002709738393609348, 0.0003188684989508893, 0.0002550965733229793, 0.0002448663446686086, 0.00024093355243655855, 0.0002555353974679931, 0.00029177361706486855, 0.0003048328870649025, 0.0002351029800437518, 0.0002248668189001782]


test_r2_ave: 0.3847482641416512
[0.3989872129365519, 0.1713147656018501, 0.45941764367971494, 0.509674842213582, 0.444323672953874, 0.3254526260874956, 0.27052174606463075, 0.31486274716295537, 0.5030653545502293, 0.4498620301656273]


In [11]:
ave(skew_data[title],y,0.4,stack_mod,0.3,vote_mod,0.3,lasso_mod)

test_rmse_ave: 0.0002639831802963029
[0.000271617066690579, 0.000319797748887557, 0.00025464587895965496, 0.0002473778285135431, 0.00024150647112400624, 0.0002519600449688757, 0.0002884024111011452, 0.00030383325550980655, 0.00023742017934105436, 0.00022327091786680727]


test_r2_ave: 0.38668687980250305
[0.3961305040768368, 0.16647780142172264, 0.46132611460379147, 0.49956517282583035, 0.44167783254241977, 0.34419658534699327, 0.28728141279912545, 0.3193488894587351, 0.493221402373198, 0.4576430825763773]


In [12]:
name='Input_C_046'
test_pred=[]
seed=[9,8,3,4,2,1,6]
stack_w=0.4
vote_w=0.3
model_w=0.3
model=lasso_mod
for i in seed:
    X_train, X_test, y_train, y_test = train_test_split(skew_data[title], y, test_size=0.2, random_state=i)

    stack = stack_mod.fit(X_train, y_train)
    stack_pred = stack.predict(test)
    vote = vote_mod.fit(X_train, y_train)
    vote_pred = vote.predict(test)
    mod = model.fit(X_train, y_train)
    mod_pred = mod.predict(test)

    ###
    final_test = (vote_w*vote_pred+stack_w*stack_pred + model_w*mod_pred)
    ###
    test_pred.append(final_test)


final_test = pd.DataFrame(np.mean(test_pred, axis=0), columns=[name])
final_test.to_csv(('./output/'+name+'.csv'))
